# 初始化

In [1]:
import os
import cv2
import gc
import time
from keras.applications.imagenet_utils import preprocess_input
import numpy as np  
np.set_printoptions(threshold=np.inf)
import tensorflow as tf
from keras import backend as K
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.Session(config=config)
K.set_session(sess)

from keras import layers, activations
from keras.models import Model
from keras.layers import Input,Dense,Flatten,Dropout,ZeroPadding2D,BatchNormalization,Activation,Add,Dot,AveragePooling2D,Lambda
from keras.layers.pooling import GlobalAveragePooling2D
from keras.layers.convolutional import Conv2D,MaxPooling2D  

Using TensorFlow backend.


# 读取训练数据

In [2]:
def read_image(image_name):
    im=cv2.imread(image_name)
    im=cv2.resize(im, (224, 224))
    return im

#数据扩充
def img_Rotation(img,angel):
    if(0 == angel):
        dst = img
    else:
        rows,cols=img.shape[:2]
        #angel度旋转
        M=cv2.getRotationMatrix2D((cols/2,rows/2),angel,1)
        dst=cv2.warpAffine(img,M,(cols,rows))
    
    return dst

In [3]:
dataset = 'AID'
Extension='.jpg'

base_path='./Dataset/' + dataset

All_Labels = os.listdir(os.path.join(base_path, 'Train'))
num_classes = len(All_Labels)
images_Train = []
labels_Train = []
images_Val = []
labels_Val = []

for i in range(0, num_classes):
    file_dir = os.path.join(base_path, 'Train', All_Labels[i])
    file_names = os.listdir(file_dir)
    for file_name in file_names:
        if(file_name.endswith(Extension)):
            finalFileName = os.path.join(file_dir, file_name)
            Label = np.linspace(0, 0, num_classes, dtype='int32')
            Label[i] = 1
            original_img = read_image(finalFileName)
            flipped_img = cv2.flip(original_img, 0)
            
            for j in [0, 45, 90, 135, 180, 225, 270, 315]: 
                images_Train.append(img_Rotation(original_img, j))
                labels_Train.append(Label)               
                images_Train.append(img_Rotation(flipped_img, j))
                labels_Train.append(Label)               
                
print(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()) + "Get %d Train Images" %(len(images_Train)))

2021-08-20 10:57:37Get 16000 Train Images


In [4]:
print(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()) + " np.array")
X_Train = np.array(images_Train, dtype='float32')
Y_Train = np.array(labels_Train)
del images_Train
gc.collect()

print(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()) + " preprocess")
X_Train = preprocess_input(X_Train)

gc.collect()
print(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()) + " Done!")

2021-08-20 10:57:37 np.array
2021-08-20 10:58:34 preprocess
2021-08-20 11:01:51 Done!


In [5]:
ModelName = "Gabor-ResNet50"
batch_size = 20
epochs = 50  
num_classes = Y_Train.shape[1]
#num_classes = 19

# Gabor滤波器

In [6]:
import cv2
def Gabor(shape, dtype=None):    
    ksize = (7, 7)
    # 核尺寸
    sigmas = [1] # [2, 4]
    # 角度
    thetas = np.linspace(0, 2*np.pi, 8, endpoint=False) # np.linspace(0, np.pi, 4, endpoint=False)
    # 波长(间隔)
    lambdas = np.linspace(2, 3, 6) # [8, 16, 32, 64]
    # 高度(越小，核函数图像会越高)
    gammas = [1] # np.linspace(1, 0, 2, endpoint=False)
    # 中轴
    psis = [0, 2*np.pi]
    
    gabors = []
    
    for i in range(0,int(64/4)):    
#     size, sigma, theta, lambda, gamma aspect ratio                 
        gf = cv2.getGaborKernel(ksize=ksize, sigma=1, theta=0, lambd=2, gamma=1, psi=0, ktype=cv2.CV_32F)
        gabors.append(gf)             
        gf = cv2.getGaborKernel(ksize=ksize, sigma=1, theta=np.pi/2, lambd=2, gamma=1, psi=0, ktype=cv2.CV_32F)
        gabors.append(gf)             
        gf = cv2.getGaborKernel(ksize=ksize, sigma=1, theta=np.pi/4, lambd=2, gamma=1, psi=0, ktype=cv2.CV_32F)
        gabors.append(gf)             
        gf = cv2.getGaborKernel(ksize=ksize, sigma=1, theta=np.pi/4*3, lambd=2, gamma=1, psi=0, ktype=cv2.CV_32F)
        gabors.append(gf)
    stacked_list = np.array([gabors])
    stacked_list = np.einsum('hijk->jkhi', stacked_list)
    
    b = K.constant(stacked_list, dtype='float32')
    F_0 = Lambda(lambda x: K.cast(x, dtype='float32'))(b)
    return F_0

# 原生ResNet-50

In [7]:
def identity_block(input_tensor, kernel_size, filters, stage, block):
    filters1, filters2, filters3 = filters
    if K.image_data_format() == 'channels_last':
        bn_axis = 3
    else:
        bn_axis = 1
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    x = Conv2D(filters1, (1, 1), name=conv_name_base + '2a')(input_tensor)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2a')(x)
    x = Activation('relu')(x)

    x = Conv2D(filters2, kernel_size,
               padding='same', name=conv_name_base + '2b')(x)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2b')(x)
    x = Activation('relu')(x)

    x = Conv2D(filters3, (1, 1), name=conv_name_base + '2c')(x)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2c')(x)

    x = layers.add([x, input_tensor])
    x = Activation('relu')(x)
    return x

def conv_block(input_tensor, kernel_size, filters, stage, block, strides=(2, 2)):
    filters1, filters2, filters3 = filters
    if K.image_data_format() == 'channels_last':
        bn_axis = 3
    else:
        bn_axis = 1
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    x = Conv2D(filters1, (1, 1), strides=strides,
               name=conv_name_base + '2a')(input_tensor)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2a')(x)
    x = Activation('relu')(x)

    x = Conv2D(filters2, kernel_size, padding='same',
               name=conv_name_base + '2b')(x)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2b')(x)
    x = Activation('relu')(x)

    x = Conv2D(filters3, (1, 1), name=conv_name_base + '2c')(x)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2c')(x)

    shortcut = Conv2D(filters3, (1, 1), strides=strides,
                      name=conv_name_base + '1')(input_tensor)
    shortcut = BatchNormalization(axis=bn_axis, name=bn_name_base + '1')(shortcut)

    x = layers.add([x, shortcut])
    x = Activation('relu')(x)
    return x 
    
from keras.optimizers import SGD  
def GaborResNet(shape):
    if K.image_data_format() == 'channels_last':
        bn_axis = 3
    else:
        bn_axis = 1
        
    img_input = Input(shape=shape)  

    x = ZeroPadding2D((3, 3))(img_input)
    x = Conv2D(64, (7, 7), strides=(2, 2), name='conv1')(x)
    x = BatchNormalization(axis=bn_axis, name='bn_conv1')(x)
    x = Activation('relu')(x)
    x = ZeroPadding2D(3)(x)
    x = Conv2D(64, (7, 7), strides=1, use_bias=False, kernel_initializer=Gabor, name='Gabor')(x)
    x = BatchNormalization(axis=bn_axis, name='Gabor-Batch')(x)
    x = Activation('relu', name='Gabor-relu')(x)
    x = MaxPooling2D((3, 3), strides=(2, 2))(x)

    x = conv_block(x, 3, [64, 64, 256], stage=2, block='a', strides=(1, 1))
    x = identity_block(x, 3, [64, 64, 256], stage=2, block='b')
    x = identity_block(x, 3, [64, 64, 256], stage=2, block='c')

    x = conv_block(x, 3, [128, 128, 512], stage=3, block='a')
    x = identity_block(x, 3, [128, 128, 512], stage=3, block='b')
    x = identity_block(x, 3, [128, 128, 512], stage=3, block='c')
    x = identity_block(x, 3, [128, 128, 512], stage=3, block='d')

    x = conv_block(x, 3, [256, 256, 1024], stage=4, block='a')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='b')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='c')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='d')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='e')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='f')

    x = conv_block(x, 3, [512, 512, 2048], stage=5, block='a')
    x = identity_block(x, 3, [512, 512, 2048], stage=5, block='b')
    x = identity_block(x, 3, [512, 512, 2048], stage=5, block='c')

    x = AveragePooling2D((7, 7), name='avg_pool')(x)
    x = Flatten()(x)  
    x = Dense(num_classes, activation='softmax', name='fc_RS19')(x)  

    model = Model(inputs=img_input,outputs=x, name="Gabor-ResNet50")  
    sgd = SGD(decay=0.0001,momentum=0.9)  
    model.compile(loss='categorical_crossentropy',optimizer=sgd,metrics=['accuracy'])  
    
    return model

In [8]:
K.clear_session()
tf.reset_default_graph()

In [9]:
model = GaborResNet(X_Train.shape[1:])
#model.load_weights('./models/ResNet50.h5', by_name=True)
model.load_weights('./models/2021-06-29/PatternNet/Gabor-ResNet50.h5', by_name=True)
model.summary()
#model.layers[6].trainable = False
#OutPutLayer = model.get_layer('flatten_1').output
#prediction = Dense(num_classes, activation='softmax')(OutPutLayer)
#new_model = Model(inputs=model.input, outputs=prediction, name="ResNet50")
#sgd = SGD(decay=0.0001,momentum=0.9)  
#new_model.compile(loss='categorical_crossentropy',optimizer=sgd,metrics=['accuracy'])  
#new_model.summary()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Model: "Gabor-ResNet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
__

In [10]:
history = model.fit(X_Train, Y_Train, batch_size=batch_size, epochs=epochs, shuffle=True)
#history = model.fit(X_Train, Y_Train, batch_size=batch_size, epochs=40, shuffle=True)


Epoch 1/50
16000/16000 [==============================] - 202s 13ms/step - loss: 0.3032 - accuracy: 0.9096
Epoch 2/50
16000/16000 [==============================] - 115s 7ms/step - loss: 0.0388 - accuracy: 0.9886
Epoch 3/50
16000/16000 [==============================] - 115s 7ms/step - loss: 0.0102 - accuracy: 0.9971
Epoch 4/50
16000/16000 [==============================] - 115s 7ms/step - loss: 0.0042 - accuracy: 0.9992
Epoch 5/50
16000/16000 [==============================] - 115s 7ms/step - loss: 0.0122 - accuracy: 0.9966
Epoch 6/50
16000/16000 [==============================] - 116s 7ms/step - loss: 0.0018 - accuracy: 0.9998
Epoch 7/50
16000/16000 [==============================] - 116s 7ms/step - loss: 0.0023 - accuracy: 0.9995
Epoch 8/50
16000/16000 [==============================] - 116s 7ms/step - loss: 0.0017 - accuracy: 0.9998
Epoch 9/50
16000/16000 [==============================] - 116s 7ms/step - loss: 8.2958e-04 - accuracy: 0.9999
Epoch 10/50
16000/16000 [===============

In [11]:
import time
save_folder = 'models/' + time.strftime("%Y-%m-%d", time.localtime()) + '/' + dataset
if not os.path.exists(save_folder):
    os.makedirs(save_folder)

# serialize model to JSON
#import pickle
model_json = model.to_json()
with open(os.path.join(save_folder, ModelName + ".json"), "w") as json_file:
    json_file.write(model_json)

# serialize weights to HDF5
model.save(os.path.join(save_folder, ModelName + ".h5"))
#pickle.dump(history.history, open('history/UCMerced_LandUse/AlexNet.p','wb'))
print("Saved Model to disk")

Saved Model to disk


In [12]:
import os
import time
import cv2
from keras.applications.imagenet_utils import preprocess_input

All_Labels = os.listdir(os.path.join(base_path, 'Test'))
images = []
labels = []
outputFileName = []

save_folder = 'FeatureMap/' + time.strftime("%Y-%m-%d", time.localtime()) + '/' + ModelName + '/' + dataset  +'/'

for i in range(0, num_classes):
    file_dir = os.path.join(base_path, 'Test', All_Labels[i])
    file_names = os.listdir(file_dir)
    for file_name in file_names:
        if(file_name.endswith(Extension)):
            finalFileName = os.path.join(file_dir, file_name)
            Label = np.linspace(0, 0, num_classes, dtype='int32')
            Label[i] = 1    
            images.append(read_image(finalFileName))
            labels.append(Label)
            outputFileName.append(os.path.join(save_folder, All_Labels[i], file_name).replace(Extension, ".txt"))

print("Get %d Test Images" %(len(images)))
output = np.array(images, dtype="float32")
output = preprocess_input(output)

Get 9000 Test Images


In [9]:
Count = 0
for i in range(0, len(output)):
    p=model.predict(output[i : i + 1])
    if np.argmax(p)==np.argmax(labels[i : i + 1][0]):
        Count += 1
    
print(Count/len(output))

0.9660714285714286


In [13]:
import os
import time
from keras.models import Model  
OutPutLayer = Model(inputs=model.input, outputs=model.get_layer('avg_pool').output)
print("Saved FeatureMap to disk...")
OutputFeatures=[]
for i in range(0, len(output)):
    p = OutPutLayer.predict(output[i : i + 1])
    out=np.reshape(p,p.shape[-1])
    OutputFeatures.append(out)
    print("\r当前输出：%d" %(i + 1), end= " ")

OutputFeatures = np.array(OutputFeatures, dtype="float") 
save_folder = 'FeatureMap/' + time.strftime("%Y-%m-%d", time.localtime()) + '/' + ModelName + '/' + dataset  +'/'
if not os.path.exists(save_folder):
    os.makedirs(save_folder)
np.savetxt(os.path.join(save_folder, dataset + '-Test-2048D.txt'), OutputFeatures)
print("\n保存完成！")
print(OutputFeatures.shape)

Saved FeatureMap to disk...
当前输出：9000                          
保存完成！
(9000, 2048)


In [32]:
import os
import time
from keras.models import Model  
OutPutLayer = Model(inputs=model.input, outputs=model.get_layer('avg_pool').output)
print("Saved FeatureMap to disk...")
OutputFeatures=[]
for i in range(0, len(X_Train)):
    p = OutPutLayer.predict(X_Train[i : i + 1])
    out=np.reshape(p,p.shape[-1])
    OutputFeatures.append(out)
    print("\r当前输出：%d" %(i + 1), end= " ")

OutputFeatures = np.array(OutputFeatures, dtype="float") 
save_folder = 'FeatureMap/' + time.strftime("%Y-%m-%d", time.localtime()) + '/' + ModelName + '/' + dataset  +'/'
if not os.path.exists(save_folder):
    os.makedirs(save_folder)
np.savetxt(os.path.join(save_folder, dataset + '-Train-2048D.txt'), OutputFeatures)
print("\n保存完成！")
print(OutputFeatures.shape)

Saved FeatureMap to disk...
当前输出：26880                                                                                               
保存完成！
(26880, 2048)
